In [13]:
# Import required libraries.

import pandas as pd
import numpy as np
import time

In [14]:
t_begin = time.time()
t_read = time.time()

In [15]:
# Read all datasets.

jan  = pd.read_csv("datasets/01_flight_details_january.csv")
feb  = pd.read_csv("datasets/02_flight_details_february.csv")
mar  = pd.read_csv("datasets/03_flight_details_march.csv")
apr  = pd.read_csv("datasets/04_flight_details_april.csv")
may  = pd.read_csv("datasets/05_flight_details_may.csv")
jun  = pd.read_csv("datasets/06_flight_details_june.csv")
jul  = pd.read_csv("datasets/07_flight_details_july.csv")
aug  = pd.read_csv("datasets/08_flight_details_august.csv")
sep  = pd.read_csv("datasets/09_flight_details_september.csv")
octo = pd.read_csv("datasets/10_flight_details_october.csv")
nov  = pd.read_csv("datasets/11_flight_details_november.csv")
dec  = pd.read_csv("datasets/12_flight_details_december.csv")


In [16]:
t_read_complete = time.time() - t_read

In [17]:
jan_copy = jan.copy()
feb_copy = feb.copy()
mar_copy = mar.copy()
apr_copy = apr.copy()
may_copy = may.copy()
jun_copy = jun.copy()
jul_copy = jul.copy()
aug_copy = aug.copy()
sep_copy = sep.copy()
octo_copy = octo.copy()
nov_copy = nov.copy()
dec_copy = dec.copy()

In [18]:
# Convert columns to appropriate datatypes.

def convert_variables(data_frame):
    
    t_convert = time.time()
        
    data_frame.FL_DATE = pd.to_datetime(data_frame.FL_DATE)
    
    data_frame.AIRLINE_ID = data_frame.AIRLINE_ID.astype("str")
    
    data_frame.FL_NUM = data_frame.FL_NUM.astype("str")
    
    data_frame.CRS_DEP_TIME = data_frame.CRS_DEP_TIME.astype("str").str.zfill(4).replace("2400", "2359")
    data_frame.CRS_ARR_TIME = data_frame.CRS_ARR_TIME.astype("str").str.zfill(4).replace("2400", "2359")    
    data_frame.CRS_DEP_TIME = pd.to_datetime(data_frame.CRS_DEP_TIME, format="%H%M").dt.time
    data_frame.CRS_ARR_TIME = pd.to_datetime(data_frame.CRS_ARR_TIME, format="%H%M").dt.time
    
    data_frame.DEP_TIME = data_frame.DEP_TIME.replace(np.nan, 0.0)
    data_frame.ARR_TIME = data_frame.ARR_TIME.replace(np.nan, 0.0)
    data_frame.DEP_TIME = data_frame.DEP_TIME.astype("int").astype("str").str.zfill(4).replace("2400", "2359")
    data_frame.ARR_TIME = data_frame.ARR_TIME.astype("int").astype("str").str.zfill(4).replace("2400", "2359")
    data_frame.DEP_TIME = pd.to_datetime(data_frame.DEP_TIME, format="%H%M").dt.time
    data_frame.ARR_TIME = pd.to_datetime(data_frame.ARR_TIME, format="%H%M").dt.time
    
    data_frame.DEP_DELAY_GROUP = data_frame.DEP_DELAY_GROUP.astype("int", errors="ignore")
    data_frame.ARR_DELAY_GROUP = data_frame.ARR_DELAY_GROUP.astype("int", errors="ignore")
    
    data_frame.DEP_DELAY = data_frame.DEP_DELAY.astype("int", errors="ignore")
    data_frame.ARR_DELAY = data_frame.ARR_DELAY.astype("int", errors="ignore")
    
    data_frame.CANCELLED = data_frame.CANCELLED.astype("bool")
    data_frame.DIVERTED = data_frame.DIVERTED.astype("bool")
    
    data_frame.TAIL_NUM = data_frame.TAIL_NUM.replace(np.NaN, "CANCELLED")
    data_frame["FLIGHT_ID"] = data_frame.index.astype("str").str.zfill(7) + "-" + data_frame["AIRLINE_ID"] + data_frame["TAIL_NUM"]
    
    data_frame.drop(["AIRLINE_ID", "TAIL_NUM", "Unnamed: 24"], axis=1, inplace=True)
        
    data_frame["ORIGIN_CITY"], data_frame["ORIGIN_STATE"] = data_frame.ORIGIN_CITY_NAME.str.split(", ").str
    data_frame["DEST_CITY"], data_frame["DEST_STATE"] = data_frame.DEST_CITY_NAME.str.split(", ").str
    data_frame.drop(["ORIGIN_CITY_NAME", "DEST_CITY_NAME"], inplace=True, axis=1)
    
    t_convert_complete = time.time() - t_convert
    
    return data_frame, t_convert_complete

In [19]:
def replace_00(data_frame):
    
    t_replace = time.time()
    
    data_frame.CARRIER_DELAY = data_frame.CARRIER_DELAY.replace(0.0, np.nan)
    data_frame.WEATHER_DELAY = data_frame.WEATHER_DELAY.replace(0.0, np.nan)
    data_frame.NAS_DELAY = data_frame.NAS_DELAY.replace(0.0, np.nan)
    data_frame.SECURITY_DELAY = data_frame.SECURITY_DELAY.replace(0.0, np.nan)
    data_frame.LATE_AIRCRAFT_DELAY = data_frame.LATE_AIRCRAFT_DELAY.replace(0.0, np.nan)
    
    t_replace_complete = time.time() - t_replace
    
    return data_frame, t_replace_complete

In [20]:
def flight_status(data_frame):
    
    t_status = time.time()
    
    fls = []
    for index, flight in data_frame.iterrows():
        if flight.CANCELLED:
            FLIGHT_STATUS = "Cancelled"
        elif flight.DIVERTED:
            FLIGHT_STATUS = "Diverted"
        else:
            FLIGHT_STATUS = "Normal"

        fls.append({"FLIGHT_ID": flight.FLIGHT_ID,
                    "FLIGHT_STATUS": FLIGHT_STATUS})

    fls_df = pd.DataFrame(fls)
    data_frame.drop(["CANCELLED", "DIVERTED"], axis=1, inplace=True)
    data_frame = data_frame.merge(fls_df, on="FLIGHT_ID", how="left")
    
    t_status_complete = time.time() - t_status

    return data_frame, t_status_complete

In [21]:
data_frame_list = [jan_copy, feb_copy, mar_copy, 
                   apr_copy, may_copy, jun_copy, 
                   jul_copy, aug_copy, sep_copy, 
                   octo_copy, nov_copy, dec_copy]
time_list = [0]

In [22]:
for df in data_frame_list:
    df, convert = convert_variables(df)
    df, replace = replace_00(df)
    df, status  = flight_status(df)
    
    time_list.append([convert, replace, status])
    
    print(len(time_list) - 1)

1
2
3
4
5
6
7
8
9
10
11
12


In [23]:
t_write = time.time()

In [24]:
data_frame_list = [jan_copy, feb_copy, mar_copy, 
                   apr_copy, may_copy, jun_copy, 
                   jul_copy, aug_copy, sep_copy, 
                   octo_copy, nov_copy, dec_copy]

master_df = pd.concat(data_frame_list)

In [25]:
master_df.to_csv("datasets/2018_cleaned.csv", index=False)

In [26]:
t_write_complete = time.time() - t_write
t_end = time.time() - t_begin

In [27]:
print("January   : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[1]))
print("February  : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[2]))
print("March     : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[3]))
print("April     : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[4]))
print("May       : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[5]))
print("June      : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[6]))
print("July      : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[7]))
print("August    : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[8]))
print("September : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[9]))
print("October   : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[10]))
print("November  : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[11]))
print("December  : Convert: {}\tReplace: {}\tStatus: {}".format(*time_list[12]))

print("\nTotal     :", t_end)
print("Begin at    :", time.ctime(t_begin))
print("End at      :", time.ctime())

January   : Convert: 11.253670692443848	Replace: 0.035102128982543945	Status: 59.774205923080444
February  : Convert: 10.595396757125854	Replace: 0.030005693435668945	Status: 54.58962798118591
March     : Convert: 12.686957836151123	Replace: 0.03588390350341797	Status: 65.44667482376099
April     : Convert: 12.416007041931152	Replace: 0.03599190711975098	Status: 62.84587335586548
May       : Convert: 13.028747081756592	Replace: 0.044010162353515625	Status: 65.4891996383667
June      : Convert: 13.028053998947144	Replace: 0.038055419921875	Status: 66.71432828903198
July      : Convert: 13.527225494384766	Replace: 0.03802847862243652	Status: 67.98509955406189
August    : Convert: 13.37300443649292	Replace: 0.03700757026672363	Status: 68.22245359420776
September : Convert: 12.175735473632812	Replace: 0.03900790214538574	Status: 60.99470090866089
October   : Convert: 12.653844356536865	Replace: 0.03400611877441406	Status: 63.8031120300293
November  : Convert: 12.046732664108276	Replace: 0.